In [1]:
import my_forex as myfx

In [2]:
df = myfx.fetch_candle('USD_CAD', granularity='D', count=200)

V20ConnectionError: Connection to v20 REST server at https://api-fxproactice.oanda.com:443/v3/instruments/USD_CAD/candles failed

In [ ]:
response = api.instrument.candles('USD_CAD', granularity='D', count=200)

print("Response: {} ({})".format(response.status, response.reason))

In [ ]:
j = json.loads(response.raw_body)
ohlc = pd.io.json.json_normalize(j['candles'])

In [ ]:
ohlc['date'] = pd.to_datetime(ohlc['time'])

In [ ]:
ohlc = ohlc[['date', 'mid.o', 'mid.h', 'mid.l', 'mid.c', 'volume']]

In [ ]:
ohlc.columns = ['date', 'o', 'h', 'l', 'c', 'volume']

In [ ]:
ohlc.o = pd.to_numeric(ohlc.o)
ohlc.h = pd.to_numeric(ohlc.h)
ohlc.l = pd.to_numeric(ohlc.l)
ohlc.c = pd.to_numeric(ohlc.c)

In [ ]:
ohlc.info()

In [ ]:
ohlc.head()

In [ ]:
#ohlc contains stock information
#append a sequence to ohlc
seqs=np.arange(ohlc.shape[0])
ohlc["seq"]=pd.Series(seqs)

#ohlc["date"] = pd.to_datetime(ohlc["date"])
ohlc['date']=ohlc['date'].apply(lambda x: x.strftime('%m/%d'))
#ohlc['changepercent']=ohlc['changepercent'].apply(lambda x: str(x)+"%")

ohlc['mid']=ohlc.apply(lambda x:(x['o']+x['c'])/2,axis=1)
ohlc['height']=ohlc.apply(lambda x:abs(x['c']-x['o'] if x['c']!=x['o'] else 0.001),axis=1)

In [ ]:
inc = ohlc.c > ohlc.o
dec = ohlc.o > ohlc.c
w=0.3

#use ColumnDataSource to pass in data for tooltips
sourceInc=bpl.ColumnDataSource(bpl.ColumnDataSource.from_df(ohlc.loc[inc]))
sourceDec=bpl.ColumnDataSource(bpl.ColumnDataSource.from_df(ohlc.loc[dec]))


#the values for the tooltip come from ColumnDataSource
hover = HoverTool(
    tooltips=[
        ("date", "@date"),
        ("open", "@o"),
        ("close", "@c")
    ]
)

TOOLS = [CrosshairTool(), hover, 'wheel_zoom', 'reset']
p = figure(plot_width=900, plot_height=400, tools=TOOLS)
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3

#this is the up tail
p.segment(ohlc.seq[inc], ohlc.h[inc], ohlc.seq[inc], ohlc.l[inc], color="red")
#this is the bottom tail
p.segment(ohlc.seq[dec], ohlc.h[dec], ohlc.seq[dec], ohlc.l[dec], color="green")
#this is the candle body for the red dates
p.rect(x='seq', y='mid', width=w, height='height', fill_color="red", line_color="red", source=sourceInc)
#this is the candle body for the green dates
p.rect(x='seq', y='mid', width=w, height='height', fill_color="green", line_color="green", source=sourceDec)

bpl.show(p)